# Nghiên cứu số lượng đối thủ cạnh tranh trong các TTTM
*Sep 21, 2020*
Các cửa hàng đc phân tích phải đạt các tiêu chí sau:
- Bán nội thất/đệm. Nếu ko bán đệm thì **ít nhất phải bán giường phòng ngủ**
- Nằm trong TTTM, ko tính chân đế chung cư & shophouse kđt
- Page Facebook phải có promo CTKM cho riêng từng nhãn hàng
- Page Facebook phải thông báo có CTKM mà có nhiều nhãn hàng cùng tham gia

## TLDR/Nhận xét
- Hoàng Hải Forever là đối thủ mạnh nhất trong các TTTM, với 34 CH
- Về lượng Likes trên Facebook, các FB của Aeon áp đảo về số lượng likes
- Một số CH của đối thủ đặt ở các 'Mart'/'Grocery Store', đây là những địa điểm tập trung chỉ bán đồ dùng thường ngày như siêu thị
- Đối với TTTM BigC:

> Các Page của từng TTTM BigC nhỏ lẻ đều thuộc Page mẹ chính

> Cách BigC đăng trên Facebook chỉ tập trung vào phần siêu thị của BigC, ko promo các gian hàng

## Chuẩn bị dữ liệu & tính các chỉ số

In [ ]:
#Raw Data
!wget https://raw.githubusercontent.com/hmvuanem/pyscripts/master/mall_analysis/Comp_Locations.xlsx

In [17]:
import pandas as pd
import plotly.graph_objects as go

In [18]:
data = pd.read_excel('Comp_Locations.xlsx', sheet_name='Data')
print(data.head())
dim = pd.read_excel('Comp_Locations.xlsx', sheet_name='Dim')
print(dim.head())

Comp             Mall
0  Akemi UCHI  Aeon Bình Dương
1  Akemi UCHI    Aeon Bình Tân
2  Akemi UCHI     Aeon Hà Đông
3  Akemi UCHI     Aeon Tân Phú
4  Akemi UCHI    Crescent Mall
              Location  Likes  Promotion  Special  Parent       Brand  \
0  Sun Plaza Thuỵ Khuê    0.0          0        0       0     Ba Đình   
1      Vincom An Giang    0.0          1        1       1  Long Xuyên   
2       Vincom Bắc Kạn    0.0          1        1       1     Bắc Kạn   
3       Vincom Bảo Lộc    0.0          1        1       1     Bảo Lộc   
4      Vincom Biên Hoà    0.0          1        1       1    Biên Hoà   

   Province  Unnamed: 7  Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  
0        HN         NaN         NaN         NaN          NaN          NaN  
1  An Giang         NaN         NaN         NaN          NaN          NaN  
2   Bắc Kạn         NaN         NaN         NaN          NaN          NaN  
3  Lâm Đồng         NaN         NaN         NaN          NaN          NaN  
4  Đ

In [19]:
df = data.merge(dim, how='left', left_on='Mall', right_on='Location').drop('Mall', axis=1)

In [20]:
allow_promotions = (df['Promotion']==1) & (df['Special']==1)
vincom = df['Location'].str.split(' ', expand=True)[0]=='Vincom'
df = df[allow_promotions]

In [21]:
stores_by_comp = df.groupby('Comp')['Location'].count().sort_values(ascending=False)
stores_by_locations = df.groupby('Location')['Comp'].count().sort_values(ascending=False)
stores_by_provinces = df.groupby('Province')['Comp'].count().sort_values(ascending=False)
mall_by_provinces = df.groupby('Province')['Location'].nunique().sort_values(ascending=False)

In [22]:
def plot_analytics(x, y, orientation, title, xaxis, yaxis):
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=x,
            y=y,
            orientation=orientation
        )
    )

    fig.update_layout(
        title=title,
        xaxis_title=xaxis,
        yaxis_title=yaxis,
        font=dict(
            family="Open Sans",
            color="#000000"
        ),
        yaxis={'categoryorder':'total ascending'},
        width=500,
        height=800
    )

    fig.show()

## Số lượng cửa hàng theo thương hiệu

In [23]:
plot_analytics(
    x=stores_by_comp.values,
    y=stores_by_comp.index,
    orientation='h',
    title='Số Lượng CH theo thương hiệu',
    xaxis='Số Lượng',
    yaxis='Thương Hiệu'
)

## Số lượng cửa hàng nội thất theo Mall

In [24]:
plot_analytics(
    y=stores_by_locations.index,
    x=stores_by_locations.values,
    orientation='h',
    title='Số Lượng CH nội thất theo Mall',
    xaxis='Số Lượng',
    yaxis='Mall'
)

## Số lượng TTTM theo Tỉnh/Thành Phố

In [25]:
plot_analytics(
    y=stores_by_provinces.index,
    x=stores_by_provinces.values,
    orientation='h',
    title='Số Lượng TTTM theo Tỉnh/Thành Phố',
    xaxis='Số Lượng',
    yaxis='Tỉnh/Thành Phố'
)

## Số lượng Likes trên Facebook theo TTTM

In [42]:
plot_analytics(
    x=dim[dim['Likes']>1e4].sort_values('Likes', ascending=False)['Likes'],
    y=dim[dim['Likes']>1e4].sort_values('Likes', ascending=False)['Location'],
    orientation='h',
    title='Số lượng Likes trên Facebook theo TTTM',
    xaxis='Số Lượng',
    yaxis='TTTM'
)